# Recaclulating Collision Meshes

In this notebook we show how we can recalculate the collision meshes of the UR5e forearm and upper arm links.
We do this by cropping the visualisation meshes into three parts and computing their convex hulls.

This notebook required the following packages to be installed:
- `open3d`
- `airo-drake`

## 1. The original URDF

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import SingleArmScene, add_floor, add_meshcat, finish_build
import airo_models

def create_single_arm_scene(urdf_path: str) -> SingleArmScene:
    robot_diagram_builder = RobotDiagramBuilder()

    meshcat = add_meshcat(robot_diagram_builder)

    plant = robot_diagram_builder.plant()
    parser = robot_diagram_builder.parser()

    # Load URDF files
    arm_index = parser.AddModels(urdf_path)[0]

    # Weld some frames together
    world_frame = plant.world_frame()
    arm_frame = plant.GetFrameByName("base_link", arm_index)

    plant.WeldFrames(world_frame, arm_frame)

    robot_diagram, context = finish_build(robot_diagram_builder)

    scene = SingleArmScene(robot_diagram, arm_index, None, meshcat)
    return scene


original_arm_urdf_path = airo_models.get_urdf_path("ur5e")
scene = create_single_arm_scene(original_arm_urdf_path)

## 2. Lets make a copy to work with

Because we store the paths to meshes as relative paths to URDFs, it's easiest to copy the entire URDF to a new directory and work with that.

In [ ]:
import os
import shutil

original_arm_dir = os.path.dirname(original_arm_urdf_path)
output_dir = "output"

os.makedirs(output_dir, exist_ok=True)

arm_name = os.path.basename(original_arm_dir)

shutil.copytree(original_arm_dir, os.path.join(output_dir, arm_name), dirs_exist_ok=True)

In [ ]:
!ls $output_dir

In [ ]:
new_arm_urdf_path = os.path.join(output_dir, arm_name, f"{arm_name}.urdf")
new_arm_urdf_dir = os.path.dirname(new_arm_urdf_path)

new_arm_urdf_dir

In [ ]:
import airo_models
from airo_models.urdf import read_urdf

arm_urdf = read_urdf(new_arm_urdf_path)


In [ ]:
from airo_models.urdf import get_link_by_name

link_name = "forearm_link"

link = get_link_by_name(arm_urdf, "forearm_link")
link

In [ ]:
visual_mesh_rel_path = link["visual"]["geometry"]["mesh"]["@filename"]
visual_mesh_rel_path

In [ ]:

from airo_models.urdf import make_path_absolute

visual_mesh_path =  make_path_absolute(visual_mesh_rel_path, new_arm_urdf_path)
visual_mesh_path

In [ ]:
import open3d as o3d

visual_mesh = o3d.io.read_triangle_mesh(visual_mesh_path)
visual_mesh

### Creating bounding boxes to crop the mesh

In [ ]:
bbox_x = 0.08
bbox_y = 0.08

z_elbow_shaft = 0.09
z_shaft_wrist = 0.353


def create_z_crop_bbox(
    z_min: float, z_max: float, size_xy: float = 0.08, color: tuple[float, float, float] = (0.0, 0.0, 1.0)
) -> o3d.geometry.AxisAlignedBoundingBox:
    """Creates a bounding box for cropping a mesh along the Z-axis.

    Args:
        z_min: The minimum Z-coordinate of the bounding box.
        z_max: The maximum Z-coordinate of the bounding box.
        size_xy: The size of the bounding box in the X and Y dimensions.
            (Defaults to 0.08)
        color: The color of the bounding box as an RGB tuple.
            (Defaults to blue)

    Returns:
        open3d.geometry.AxisAlignedBoundingBox
    """

    bbox = ((-size_xy, -size_xy, z_min), (size_xy, size_xy, z_max))
    bbox_o3d = o3d.geometry.AxisAlignedBoundingBox(*bbox)
    bbox_o3d.color = color
    return bbox_o3d


z_ranges_forearm = [(-0.08, 0.09), (0.09, 0.353), (0.353, 0.45)]
z_range_upperarm = [(-0.08, 0.09), (0.09, 0.353), (0.353, 0.45)]
z_ranges = {"forearm_link": z_ranges_forearm, "upperarm": z_range_upperarm}


z_range = z_ranges[link_name]

bboxes = [create_z_crop_bbox(*z) for z in z_range]


visual_mesh.compute_vertex_normals()
world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])


o3d.visualization.draw_geometries([visual_mesh, *bboxes, world_frame])

### Cropping and calculating the convex hulls

In [ ]:
hulls = []
crops = []


def create_hulls_of_crops(
    mesh: o3d.geometry.TriangleMesh, bboxes: list[o3d.geometry.AxisAlignedBoundingBox]
) -> list[o3d.geometry.TriangleMesh]:
    """Creates the convex hulls of the crops of a mesh.

    Args:
        mesh: The mesh to crop and create convex hulls from.
        bboxes: The bounding boxes to use for cropping.

    Returns:
        list[o3d.geometry.TriangleMesh]
    """

    crops = [mesh.crop(bbox) for bbox in bboxes]
    hulls = [crop.compute_convex_hull()[0] for crop in crops]

    for hull in hulls:
        hull.compute_vertex_normals()

        color = np.random.rand(3)
        hull.paint_uniform_color(color)

    return hulls


hulls = create_hulls_of_crops(visual_mesh, bboxes)

o3d.visualization.draw_geometries([*hulls, world_frame])

### Replacing the collision element

In [ ]:
link["collision"]

In [ ]:
origin = link["collision"]["origin"].copy()
origin

In [ ]:
import tempfile



def create_new_collision_elements(hulls: list[o3d.geometry.TriangleMesh], origin: dict, collision_mesh_dir: str, link_name: str, urdf_dir: str = None):
    new_collision_elements = []

    
    for i, hull in enumerate(hulls):
        hull_obj_path = os.path.join(collision_mesh_dir, f"{link_name}_part_{i}.obj")

        print(hull_obj_path)

        hull_obj_rel_path = os.path.relpath(hull_obj_path, urdf_dir)

        o3d.io.write_triangle_mesh(hull_obj_path, hull)

        collision = {
            "origin": origin,
            "geometry": {
                "mesh": {
                    "@filename": hull_obj_rel_path
                }
            }
        }

        new_collision_elements.append(collision)

    return new_collision_elements


# TODO make sure the meshes are written to the right location, but the paths in the URDF are relative to the URDF file
collision_mesh_dir = os.path.join(new_arm_urdf_dir, "collision")
new_collision_elements =create_new_collision_elements(hulls, origin, collision_mesh_dir, link_name, new_arm_urdf_dir)

new_collision_elements

### Editing and resaving the URDF

In [ ]:
from airo_models.urdf import write_urdf_to_file

link["collision"] = new_collision_elements

write_urdf_to_file(arm_urdf, new_arm_urdf_path)

In [ ]:
scene = create_single_arm_scene(new_arm_urdf_path)